<a href="https://colab.research.google.com/github/TirendazAcademy/Awesome-Data-Science-Resources/blob/main/Introduction%20to%20PyTorch%20Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is PyTorch-Lightning

- PyTorch Lightning is the deep learning framework for high-performance AI research. It was designed to help researchers and developers train and evaluate deep learning models more easily and efficiently. 

- PyTorch Lightning makes it easier to write clean, modular code for training and deploying complex PyTorch models by abstracting out the training logic and providing useful utilities for things like distributed training, early stopping, and logging. It is especially well-suited for use in research, where fast experimentation is crucial.

# Install pytorch-lightning


In [1]:
!pip install pytorch-lightning
# !conda install pytorch-lightning -c conda-forge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import required libraries

In [2]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import pytorch_lightning as pl

# Define a LightningModule

LightningModule is the base class for PyTorch Lightning models. It is a subclass of the PyTorch Module class, which is a core part of the PyTorch framework for defining and training deep learning models. 

A LightningModule provides your PyTorch nn.Module to play together in complex ways inside the training_step (there is also an optional validation_step and test_step).

nn.Sequential is a module that can pack multiple components into a complicated or multilayer network

Let's define two Sequential number of nn.Modules (or use your current ones).

In [3]:
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

In [4]:
encoder

Sequential(
  (0): Linear(in_features=784, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=3, bias=True)
)

In [5]:
decoder

Sequential(
  (0): Linear(in_features=3, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=784, bias=True)
)

Let's define the LightningModule.

In [6]:

class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

Let's init the autoencoder

In [7]:
autoencoder = LitAutoEncoder(encoder, decoder)

# Define a dataset

Let's load our dataset with the Dataloader. Lightning supports ANY iterable (DataLoader, numpy, etc…) for the train/val/test/predict splits.

In [8]:
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

# Train the model

You can train your model in just a few lines of code with the use of Trainer — a function built on top of nested loops that run over all batches in a dataloader, over all epochs. 

The Lightning Trainer “mixes” any LightningModule with any dataset and abstracts away all the engineering complexity needed for scale.

In [9]:
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


All the following are handled by Trainer:

- Epoch and batch iteration
- optimizer.step(), loss.backward(), optimizer.zero_grad() calls
- Calling of model.eval(), enabling/disabling grads during evaluation
- Checkpoint Saving and Loading
- Tensorboard (see loggers options)
- Multi-GPU support
- TPU
- 16-bit precision AMP support


# Use the model

Once you’ve trained the model you can export to onnx, torchscript and put it into production or simply load the weights and run predictions.

In [10]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = Tensor(4, 28 * 28)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[ 0.1363, -0.0794, -0.0293],
        [ 0.1363, -0.0794, -0.0293],
        [ 0.1363, -0.0794, -0.0293],
        [ 0.1363, -0.0794, -0.0293]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


# Visualize training

Lightning comes with a lot of batteries included. A helpful one is Tensorboard for visualizing experiments.

Run this on your commandline and open your browser to http://localhost:6006/



In [11]:
!tensorboard --logdir .


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In this notebook, I talked about how to perform a simple deep learning analysis with PyTorch Lightning. That's it. I hope you enjoy it. 

Let's connect : [YouTube](https://www.youtube.com/channel/UCFU9Go20p01kC64w-tmFORw) | [Twitter](https://twitter.com/TirendazAcademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [Tiktok](https://www.tiktok.com/@tirendazacademy) | [Medium](https://tirendazacademy.medium.com) | [Reddit](https://www.reddit.com/user/TirendazAcademy) 